Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
verbose = True
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  global verbose
  if verbose:
    print(predictions.shape, labels.shape)
    verbose = False
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction, depth=vocabulary_size):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, depth], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution(depth=vocabulary_size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, depth])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.297808 learning rate: 10.000000
(640, 27) (640, 27)
Minibatch perplexity: 27.05
xe  rgituitoapeqezbsyjppmw get xetiqaitwsyfdbosmy  uinoeghdohafkdhwxuylgddlbecft
dwozxc nm evsrpoxm tnapeamd i oevmava br rbonuecg awt at xzln cyw tfzkiooie  efd
vqjfnt kcs rlt jcwearvmdvr j hzsetnbtmldremnpv zrcetrfwhsunecsteva ttcqekxpd eix
jeevnrhat ctdtg elseltu   ltlhi gbqao ag vtnhrmsjvlmjlatdh t sleku  mhxacigusxi 
 ekpl c  juhjz m su gxf ew fkqfxim dircrusaieplkaflrtfgaowh  pktehrejrriie tiwie
Validation set perplexity: 20.29
Average loss at step 100: 2.598798 learning rate: 10.000000
Minibatch perplexity: 10.96
Validation set perplexity: 10.40
Average loss at step 200: 2.250192 learning rate: 10.000000
Minibatch perplexity: 8.65
Validation set perplexity: 8.50
Average loss at step 300: 2.099673 learning rate: 10.000000
Minibatch perplexity: 7.38
Validation set perplexity: 7.96
Average loss at step 400: 2.003008 learning rate: 10.000000
Minibatch perplexity: 7.5

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # The big matrix
  Tw = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, 4 * num_nodes], -0.1, 0.1))
  Tb = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, state, cell):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    i = tf.concat([i, state], 1)
    tmp = tf.matmul(i, Tw) + Tb
    
    inputs_gate = tf.sigmoid(tmp[:, :num_nodes])
    forget_gate = tf.sigmoid(tmp[:, num_nodes:2*num_nodes])
    output_gate = tf.sigmoid(tmp[:, 2*num_nodes:3*num_nodes])
    update_gate = tf.tanh(tmp[:, 3*num_nodes:])
    
    cell = forget_gate * cell + inputs_gate * update_gate
    state = output_gate * tf.tanh(cell)
    return state, cell

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298967 learning rate: 10.000000
Minibatch perplexity: 27.08
khepybinbtjigrtx eoi y    ghs rny g h mt use qk rt unvr  ar  tqbdsweoc nzo    ti
qbitekf acgw qohxed otssmnrkl snq wxzzlbdqt igvekniv raasa b ifz   cre eeirbham 
wvqzsspcyxcgtarmyd n kgltjekgbtewlmja crvmpttiisrtrhs vearndythraip  txua s  uwb
neieeizggbo qbkhtlqo e oeyrqvrtvzze fs swa snaadnsowq dnj b  rrcg  rkckejuvira v
ddzw if r uml  l d qvbtno pr sefqedl us tadevokrnme nxejrfeyx kzbavae njcavifpd 
Validation set perplexity: 20.07
Average loss at step 100: 2.580508 learning rate: 10.000000
Minibatch perplexity: 10.52
Validation set perplexity: 10.92
Average loss at step 200: 2.242405 learning rate: 10.000000
Minibatch perplexity: 8.31
Validation set perplexity: 8.64
Average loss at step 300: 2.089513 learning rate: 10.000000
Minibatch perplexity: 6.43
Validation set perplexity: 8.15
Average loss at step 400: 2.034798 learning rate: 10.000000
Minibatch perplexity: 7.94
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [14]:
bigrams = list(set([''.join(t) for t in zip(text, text[1:])]))
n_bigrams = len(bigrams)
print('Number of bigrams: %d of 756 possibly existing' % n_bigrams)

bi2id = {}
id2bi = {}

for i, b in enumerate(bigrams):
    id2bi[i] = b
    bi2id[b] = i

Number of bigrams: 728 of 756 possibly existing


In [15]:
class BiBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // (2 * batch_size)
    self._cursor = [ offset * segment for offset in range(2 * batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
    for b in range(self._batch_size):
      batch[b] = bi2id[self._text[self._cursor[b]] + self._text[self._cursor[b] + 1]]
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

In [16]:
bi_train_batches = BiBatchGenerator(train_text, batch_size, num_unrollings)
bi_valid_batches = BiBatchGenerator(valid_text, 1, 1)

In [17]:
b = bi_train_batches.next()
tmp = ''
for i in range(len(b)):
    tmp += id2bi[b[i][0]]
tmp

'ons anarchists advocat'

In [18]:
b = train_batches.next()
tmp = ''
for i in range(len(b)):
    tmp += id2char(np.argmax(b[i][0]))
tmp

'iece of his'

In [40]:
def one_hot(values, depth):
    assert (values < depth).all(), 'Incorrect value for given depth'
    res = np.zeros([len(values), depth], dtype=np.float32)
    for i, v in enumerate(values):
        res[i][v] = 1.
    return res

In [53]:
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Embedding
  embeddings = tf.Variable(tf.truncated_normal([n_bigrams, embedding_size], -0.1, 0.1), name='embeddings')
  # The big matrix
  Tw = tf.Variable(tf.truncated_normal([embedding_size + num_nodes, 4 * num_nodes], -0.1, 0.1))
  Tb = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, n_bigrams], -0.1, 0.1))
  b = tf.Variable(tf.zeros([n_bigrams]))
  
  # Definition of the cell computation.
  def lstm_cell(i, state, cell):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    i = tf.concat([i, state], 1)
    tmp = tf.matmul(i, Tw) + Tb
    
    inputs_gate = tf.sigmoid(tmp[:, :num_nodes])
    forget_gate = tf.sigmoid(tmp[:, num_nodes:2*num_nodes])
    output_gate = tf.sigmoid(tmp[:, 2*num_nodes:3*num_nodes])
    update_gate = tf.tanh(tmp[:, 3*num_nodes:])
    
    cell = forget_gate * cell + inputs_gate * update_gate
    state = output_gate * tf.tanh(cell)
    return state, cell

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    e = tf.reshape(tf.nn.embedding_lookup(embeddings, i), [batch_size, -1])
    sparse_e = e # tf.nn.dropout(e, keep_prob=0.5)
    output, state = lstm_cell(sparse_e, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    sparse_output = tf.nn.dropout(tf.concat(outputs, 0), keep_prob=0.7)
    logits = tf.nn.xw_plus_b(sparse_output, w, b)
    true = tf.one_hot(indices=tf.concat(train_labels, 0), depth=n_bigrams)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=true, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 3000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embedding = tf.reshape(tf.nn.embedding_lookup(embeddings, sample_input), [1, -1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_embedding, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [54]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = bi_train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
            np.exp(logprob(predictions, one_hot(labels, n_bigrams)))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample_distribution(random_distribution(depth=n_bigrams)[0])
          sentence = id2bi[feed]
          reset_sample_state.run()
          for _ in range(40):
            prediction = sample_prediction.eval({sample_input: np.array([feed])})
            feed = sample_distribution(prediction[0])
            sentence += id2bi[feed]
          print('--> ' + sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = bi_valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, one_hot(b[1], n_bigrams))
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.624696 learning rate: 10.000000
Minibatch perplexity: 753.48
--> pncieukxxvvbftrviyhkrddwlqbvwklfzyvomdgaqjjhhjearjrhkdrbruouenspampssomivqikbwahqq
--> efxqrslhrlmjjmqjzqp qsjknirelomh nlncijhopedvevec  avq ckkwubrqgpciakwnumilnyomreq
--> lonqvsmv he uhtlrpoptcshwm kzqcfvdt tukgpyqozavjuisuqmquawztpjvuxyqoble iewaqqgs k
--> jaszkhdmcnriiyhwuemocphewwciovnsbfveavocbrnbxcgdlti arbynkfgmveinjwfsykbdahjwce mf
--> fzvlqemczvjdmcdlpoccsklnmn zod borejfmar vfwrsrsutgdouktoyxvqgrybbxmalrjrrdbgtocby
Validation set perplexity: 535.61
Average loss at step 100: 5.331909 learning rate: 10.000000
Minibatch perplexity: 149.29
Validation set perplexity: 137.31
Average loss at step 200: 4.667777 learning rate: 10.000000
Minibatch perplexity: 77.85
Validation set perplexity: 91.77
Average loss at step 300: 4.263877 learning rate: 10.000000
Minibatch perplexity: 64.97
Validation set perplexity: 67.85
Average loss at step 400: 4.055407 learning rate: 10.000000
Minibat

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [45]:
# Didn't have time. Idea is to tie up two LSTMs. First (encoder) "memorizes" data
# into state and outputs the latest state. Second (decoder) tries to decode the data.

In [57]:
from nltk import word_tokenize

In [58]:
tokenized = word_tokenize(text)
print(tokenized[0])

anarchism


In [137]:
print(len(tokenized))

17007698


In [463]:
valid_pct = 0.01
valid_size = int(len(tokenized) * valid_pct)
print('Using validation size %d' % valid_size)
valid_dataset = tokenized[:valid_size]
train_dataset = tokenized[valid_size:]

Using validation size 170076


In [464]:
sent_len = 4
train_sent = []
train_sent_tokenized = []
valid_sent = []
valid_sent_tokenized = []

for offset in range((len(tokenized) - valid_size) // sent_len):
    train_sent_tokenized.append(train_dataset[offset*sent_len:(offset+1)*sent_len])
    train_sent.append(' '.join(train_dataset[offset*sent_len:(offset+1)*sent_len]))
for offset in range(valid_size // sent_len):
    valid_sent_tokenized.append(valid_dataset[offset*sent_len:(offset+1)*sent_len])
    valid_sent.append(' '.join(valid_dataset[offset*sent_len:(offset+1)*sent_len]))
    
print(len(train_sent))
print(len(valid_sent))

4209405
42519


In [465]:
def mirror(w):
    res = ''
    n = len(w)
    for i in range(n):
        res += w[n-i-1]
    return res

In [466]:
train_sent_mirrored = []
valid_sent_mirrored = []

for s in train_sent_tokenized:
    train_sent_mirrored.append(' '.join([mirror(w) for w in s]))
    
for s in valid_sent_tokenized:
    valid_sent_mirrored.append(' '.join([mirror(w) for w in s]))
    
print(valid_sent[0])
print(valid_sent_mirrored[0])

anarchism originated as a
msihcrana detanigiro sa a


In [467]:
seq_len = max([max([len(train_sent[i]) for i in range(len(train_sent))]),
               max([len(valid_sent[i]) for i in range(len(valid_sent))])])
print('Max sequence length %d' % seq_len)

Max sequence length 147


In [468]:
#seq_len = 30
#train_lens = np.array([len(train_sent[i]) for i in range(len(train_sent))])
#valid_lens = np.array([len(valid_sent[i]) for i in range(len(valid_sent))])
#train_ind = np.where(train_lens <= seq_len)[0]
#valid_ind = np.where(valid_lens <= seq_len)[0]
#
#train_sent = np.array(train_sent)[train_ind]
#train_sent_mirrored = np.array(train_sent_mirrored)[train_ind]
#valid_sent = np.array(valid_sent)[valid_ind]
#valid_sent_mirrored = np.array(valid_sent_mirrored)[valid_ind]
#
#print('Number of train sentences: %d\nNumber of valid sentences: %d' % (len(train_ind), len(valid_ind)))

In [469]:
encoder_num_unrollings = 10

class EncoderBatchGenerator(object):
  def __init__(self, sentences, batch_size, num_unrollings):
    self._sentences = sentences
    self._n_sentences = len(sentences)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    self._lengths = np.array([len(s) for s in sentences])
    self._idx_cursor = 0
    self._str_cursor = 0
    self._last_batch = self._next_batch()
    self._str_cursor = 1
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      if self._idx_cursor + b < self._n_sentences: 
        sent_idx = self._idx_cursor + b
        if self._str_cursor < len(self._sentences[sent_idx]):
          batch[b, char2id(self._sentences[sent_idx][self._str_cursor])] = 1.0
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
      self._str_cursor = self._str_cursor + 1
    if (self._str_cursor >= self._lengths).all():
        self._str_cursor = 0
        self._idx_cursor = (self._idx_cursor + 1) % self._n_sentences
    self._last_batch = batches[-1]
    return batches

In [470]:
en_train_batches = EncoderBatchGenerator(train_sent, batch_size, encoder_num_unrollings)
en_valid_batches = EncoderBatchGenerator(valid_sent, 1, encoder_num_unrollings)

In [425]:
b = en_train_batches.next()
tmp = [''] * len(b[0])
for j in range(len(b[0])):
    for i in range(len(b)):
        tmp[j] += id2char(np.argmax(b[i][j]))
tmp[0]

'one nine fi'

In [439]:
b = en_valid_batches.next()
tmp = [''] * len(b[0])
for j in range(len(b[0])):
    for i in range(len(b)):
        tmp[j] += id2char(np.argmax(b[i][j]))
tmp

['anarchism o']

In [471]:
de_train_batches = EncoderBatchGenerator(train_sent_mirrored, batch_size, encoder_num_unrollings)
de_valid_batches = EncoderBatchGenerator(valid_sent_mirrored, 1, 1)

In [418]:
b = de_train_batches.next()
tmp = [''] * len(b[0])
for j in range(len(b[0])):
    for i in range(len(b)):
        tmp[j] += id2char(np.argmax(b[i][j]))
tmp[0]

'eno enin ev'

In [ ]:
graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Encoder
  # The big matrix
  Tw_one = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, 4 * num_nodes], -0.1, 0.1))
  Tb_one = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  # Variables saving state across unrollings.
  saved_output_one = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state_one = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Decoder
  # The big matrix
  Tw_two = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, 4 * num_nodes], -0.1, 0.1))
  Tb_two = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  # Variables saving state across unrollings.
  saved_output_two = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state_two = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, state, cell, which='one'):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    assert which in ['one', 'two'], 'Improper LSTM'
    i = tf.concat([i, state], 1)
    if which is 'one':
        tmp = tf.matmul(i, Tw_one) + Tb_one
    else:
        tmp = tf.matmul(i, Tw_two) + Tb_two
    
    inputs_gate = tf.sigmoid(tmp[:, :num_nodes])
    forget_gate = tf.sigmoid(tmp[:, num_nodes:2*num_nodes])
    output_gate = tf.sigmoid(tmp[:, 2*num_nodes:3*num_nodes])
    update_gate = tf.tanh(tmp[:, 3*num_nodes:])
    
    cell = forget_gate * cell + inputs_gate * update_gate
    state = output_gate * tf.tanh(cell)
    return state, cell

  # Input data.
  train_inputs = []
  train_labels = []
  for _ in range(num_unrollings + 1):
    train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))

  # Unrolled LSTM loop.
  output_one = saved_output_one
  state_one = saved_state_one
  for i in train_inputs:
    output_one, state_one = lstm_cell(i, output_one, state_one)
    
  output_two = output_one
  state_two = tf.zeros_like(state_one)
  outputs = []
  for i in train_labels:
    output_two, state_two = lstm_cell(i, output_two, state_two, 'two')
    outputs.append(output_two)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output_one.assign(output_one),
                                saved_state_one.assign(state_one),
                                saved_output_two.assign(output_two),
                                saved_state_two.assign(state_two)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_inputs = []
  sample_labels = []
  for _ in range(num_unrollings + 1):
    sample_inputs.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
    
  saved_sample_output_one = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state_one = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output_two = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state_two = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output_one.assign(tf.zeros([1, num_nodes])),
    saved_sample_state_one.assign(tf.zeros([1, num_nodes])), 
    saved_sample_output_two.assign(tf.zeros([1, num_nodes])),
    saved_sample_state_two.assign(tf.zeros([1, num_nodes])))

  sample_output_one = saved_sample_output_one
  sample_state_one = saved_sample_state_one
  for i in sample_inputs:
    sample_output_one, sample_state_one = lstm_cell(i, sample_output_one, sample_state_one)
    
  sample_outputs = []
  sample_output_two = sample_output_one
  sample_state_two = tf.zeros_like(sample_state_one)
  for i in sample_labels:
    sample_output_two, sample_state_two = lstm_cell(i, sample_output_two, sample_state_two)
    sample_outputs.append(sample_output_two)
    
  with tf.control_dependencies([saved_sample_output_one.assign(sample_output_one),
                                saved_sample_state_one.assign(sample_state_one), 
                                saved_sample_output_two.assign(sample_output_two),
                                saved_sample_state_two.assign(sample_state_two)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(tf.reshape(sample_outputs, [-1, num_nodes]), w, b))

In [ ]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    f_batches = en_train_batches.next()
    f_labels = de_train_batches.next()
    feed_dict = {}
    for i in range(num_unrollings + 1):
      feed_dict[train_inputs[i]] = f_batches[i]
      feed_dict[train_labels[i]] = f_labels[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      print('Minibatch perplexity: %.2f\nPlease be patient: validation is huge' % float(
        np.exp(logprob(predictions, np.reshape(f_labels, [-1, vocabulary_size])))))
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        f_b = en_valid_batches.next()
        f_l = de_valid_batches.next()
        valid_feed = {}
        for i in range(num_unrollings + 1):
            valid_feed[sample_inputs[i]] = f_b[i]
        predictions = sample_prediction.eval(valid_feed)
        valid_logprob = valid_logprob + logprob(predictions, np.array(f_l))
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295932 learning rate: 10.000000
Minibatch perplexity: 27.00
Please be patient: validation is huge
Validation set perplexity: 1.00
Average loss at step 100: 4.016036 learning rate: 10.000000
Minibatch perplexity: 1.00
Please be patient: validation is huge
